# Takes a sample of patients, extracts  and runs the stored POC model on them, then writes results to hdfs.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import datetime as datetime
from sklearn.ensemble import GradientBoostingClassifier
from impala.util import as_pandas
from sklearn.externals import joblib
import cPickle as pickle
%matplotlib notebook
plt.style.use('ggplot')

In [2]:
from impala.dbapi import connect
conn = connect(host="socbddn01.sharp.com", port=21050)
cur = conn.cursor()
cur.execute("use poc_sandbox")

In [3]:
def pull_and_writedata_2query(masterdf, i, timeoffset):
    '''
    Input: masterdf to write,  index, timeoffset in hours
    returns: masterdf with row values filled out
    '''
    enc = masterdf.encntr_id.iloc[i]
    timestart = masterdf.encntr_starttime.iloc[i] + timeoffset*3600*1000
    timeend = timestart + 12*3600*1000
    age = masterdf.age.iloc[i]
    
#     print "timestart: {0}; timeend: {1}".format(timestart, timeend)

    
    query = '''
    SELECT ce.encntr_id, ce.event_cd 
    ,cv_event_cd.description AS event_description
    ,ce.event_end_dt_tm AS unix_event_end_tm 
    , ce.result_val, ce.result_units_cd
    ,tc.checkin_dt_tm AS checkin_time
    FROM clinical_event ce 
    JOIN encounter enc ON enc.encntr_id = ce.encntr_id
    JOIN tracking_item ti ON enc.encntr_id = ti.encntr_id
    JOIN tracking_checkin tc ON tc.tracking_id = ti.tracking_id
    LEFT OUTER JOIN code_value   cv_event_cd 
    ON   ce.event_cd           = cv_event_cd.code_value     
    WHERE ce.encntr_id = '{0}' 
    AND ce.event_end_dt_tm < {1} 
    AND ce.result_status_cd NOT IN ('31', '36')
    AND ce.event_class_cd NOT IN ('654645')
    AND ce.valid_until_dt_tm > 4e12
    AND ce.event_cd IN ('679984', '2797130','2798305', '703306', '703501', '703511', '703516', 
    '703540', '703558', '2700653',
    '3623994', '4674677', '4686698', '679984', '2797130','2798305', '2797129', '75144985',
    '54411998', '2700653', '4674677', '3618608', '186470117') 
    ORDER BY ce.encntr_id, ce.performed_dt_tm;
    '''.format(enc, timeend)

#         AND ce.event_end_dt_tm > tc.checkin_dt_tm
    
    cur.execute(query)
    df = as_pandas(cur)
    df['result_val'] = pd.to_numeric(df.result_val, errors = 'coerce')

    df_timebox = df[df['unix_event_end_tm']>timestart]
    
    pairs = [('DBP', '703516'), ('SBP', '703501'),
          ('MAP', '703306'), ('temp', '703558'), ('RR', '703540'),
          ('SPO2', '3623994'), ('pulse', '703511') ]

    for pair in pairs:       
        if (df_timebox[df_timebox['event_cd']==pair[1]]).empty: # df with this event code is empty
            masterdf.ix[i, pair[0]+"_mean"] = np.nan
            masterdf.ix[i, pair[0]+"_recent"] = np.nan
        else:
            masterdf.ix[i, pair[0]+"_mean"] = df_timebox[df_timebox['event_cd']==pair[1]]['result_val'].mean()
            masterdf.ix[i, pair[0]+"_recent"] = df_timebox[df_timebox['event_cd']==pair[1]
                                                  ].sort_values(by='unix_event_end_tm', ascending=False).iloc[0]['result_val']
    
#             print "For {0}: mean = {1}, recent = {2}".format(pair[0],df_timebox[df_timebox['event_cd']==pair[1]]['result_val'].mean(),
#                                                            df_timebox[df_timebox['event_cd']==pair[1]
#                                                   ].sort_values(by='unix_event_end_tm', ascending=False).iloc[0]['result_val'] )
    

    if (df[df['event_cd']=='679984']).empty: 
        masterdf.ix[i, 'on_iv'] = 0
    else:
        masterdf.ix[i, 'on_iv'] = 1
        
    if ( (df[df['event_cd']=='2797130']).empty &
         (df[df['event_cd']=='2798305']).empty &
         (df[df['event_cd']=='2797129']).empty ):
        masterdf.ix[i, 'bu-nal'] = 0
    else:
        masterdf.ix[i, 'bu-nal'] = 1
        
    if df[df['event_cd']=='186470117'].empty:
        masterdf.ix[i, 'dialysis'] = 0
    else:
        masterdf.ix[i, 'dialysis'] = 1
        
    # smoking status - binary, 0 for nonsmoker/former smoker /unknown, 1 for smoker
    # assuming if more than 1 of these smoking lines exist, they will say the same thing.
    if (df[df.event_cd=='75144985']).empty:
        masterdf.ix[i, 'smoker'] = 0
    elif ( (df[df.event_cd=='75144985'].result_val.get_values()[0] == 'Heavy tobacco smoker') | 
     (df[df.event_cd=='75144985'].result_val.get_values()[0] == 'Light tobacco smoker') |
     (df[df.event_cd=='75144985'].result_val.get_values()[0] == 'Current every day smoker') |
     (df[df.event_cd=='75144985'].result_val.get_values()[0] == 'Current some day smoker') |
     (df[df.event_cd=='75144985'].result_val.get_values()[0] == 'Smoker, current status unknown')  ):
        masterdf.ix[i, 'smoker'] = 1
    else: 
        masterdf.ix[i, 'smoker'] = 0
        
    # previous RRT event
    if (df[df.event_cd=='54411998']).empty:
        masterdf.ix[i, 'prev_rrt'] = 0
    else:
        masterdf.ix[i, 'prev_rrt'] = 1
        
    # Obesity status
    if (df[df['event_cd']=='2700653']).empty:
        masterdf.ix[i, 'obese'] = np.nan
    elif (df[df['event_cd']=='4674677']).empty: 
        masterdf.ix[i, 'obese'] = np.nan
    else:
        # assuming there won't be much variation -- grab first value from both of height & weight
        height = pd.to_numeric(df[df['event_cd']=='2700653']['result_val'].get_values())[0]
        if df[df['event_cd']=='2700653']['result_units_cd'].get_values()[0] == '267':
            # convert inch -> cm
            height = height * 2.54
        height = height/100.0  # convert to get height in m
        weight = pd.to_numeric(df[df['event_cd']=='4674677']['result_val'].get_values())[0]
        bmi = weight / (height*height)
        if (bmi>30) & (age>19):
            masterdf.ix[i, 'obese'] = 1
        else:
            masterdf.ix[i, 'obese'] = 0
          
    
    # querying orders for medications
    query_ords = '''SELECT ords.encntr_id, mdx.multum_category_id, orig_order_dt_tm
    FROM (SELECT encntr_id, cki, substr(cki,9) as cki_id, order_id, orig_order_dt_tm FROM orders) ords 
    LEFT OUTER JOIN mltm_category_drug_xref mdx ON ords.cki_id = mdx.drug_identifier 
    LEFT OUTER JOIN mltm_drug_categories mdc ON mdc.multum_category_id = mdx.multum_category_id 
    WHERE mdx.multum_category_id IN ('261', '262','285', '283', '60', '191', '77', '210', '251', '341', '20', '21', 
                                   '22', '23', '24', '25', '26') 
    AND ords.encntr_id = '{0}'
    AND ords.orig_order_dt_tm < {1} 
    ;'''.format(enc, timeend)
    
    cur.execute(query_ords)
    df_ords = as_pandas(cur)
    
    if df_ords[(df_ords['multum_category_id']=='261') | (df_ords['multum_category_id']=='262') 
        | (df_ords['multum_category_id']=='283') | (df_ords['multum_category_id']=='285') ].empty:
        masterdf.ix[i, 'anticoagulants'] = 0
    else:
        masterdf.ix[i, 'anticoagulants'] = 1

    if (df_ords[df_ords['multum_category_id']=='60']).empty: # df with this event code is empty
        masterdf.ix[i, 'narcotics'] = 0
    else:
        masterdf.ix[i, 'narcotics'] = 1

    if (df_ords[df_ords['multum_category_id']=='191']).empty: # df with this event code is empty
        masterdf.ix[i, 'narc-ans'] = 0
    else:
        masterdf.ix[i, 'narc-ans'] = 1

    if df_ords[ (df_ords['multum_category_id']=='77') | (df_ords['multum_category_id']=='210') 
        | (df_ords['multum_category_id']=='251') | (df_ords['multum_category_id']=='341') ].empty:
        masterdf.ix[i, 'antipsychotics'] = 0
    else:
        masterdf.ix[i, 'antipsychotics'] = 1

    if df_ords[ (df_ords['multum_category_id']=='20') | (df_ords['multum_category_id']=='21') 
        | (df_ords['multum_category_id']=='22') | (df_ords['multum_category_id']=='23') 
        | (df_ords['multum_category_id']=='24') | (df_ords['multum_category_id']=='25')
        | (df_ords['multum_category_id']=='26') ].empty:
        masterdf.ix[i, 'chemo'] = 0
    else:
        masterdf.ix[i, 'chemo'] = 1
        
#     print "timebox mean of DBP: {0}".format(df_timebox[df_timebox['event_cd']=='703516']['result_val'].mean())
#     print df_timebox[df_timebox['event_cd']=='703516']
#     print "timebox_min: {0}    timebox_max: {1}".format(df_timebox['unix_event_end_tm'].min(),df_timebox['unix_event_end_tm'].max() )   # print the times used for the entries
    return masterdf

### Patient selection

In [4]:
# pull encounter ids, encounter start times, patient info for the right kind of patients from the right hospital who have stays > 36 hours
# using encounter arrive time rather than tc.checkin_dt_tm -- tc.checkin_dt_tm may have multiple values per checkin time.

query = '''
SELECT enc.encntr_id
, enc.arrive_dt_tm as encntr_starttime
, (enc.depart_dt_tm - enc.arrive_dt_tm)/3600000 as encntr_duration_hrs
, enc.depart_dt_tm as encntr_endtime
, from_unixtime(CAST(enc.encntr_complete_dt_tm/1000 as bigint)) AS encntr_complete_time
, year(now()) - year(from_unixtime(CAST(p.birth_dt_tm/1000 as bigint))) AS age 
, CASE p.sex_cd WHEN '362' then 'F' ELSE 'M' END as sex
, cvr.description as race
FROM encounter enc
INNER JOIN person p on p.person_id = enc.person_id
LEFT OUTER JOIN code_value cvr ON cvr.code_value = p.race_cd
WHERE enc.depart_dt_tm - enc.arrive_dt_tm > 3600*1000*36
AND enc.admit_type_cd != '0'
AND enc.encntr_type_class_cd = '391'
AND enc.loc_facility_cd='633867'
AND enc.encntr_complete_dt_tm < 4e12
ORDER BY enc.depart_dt_tm DESC
LIMIT 10
'''

cur.execute(query)
dfshell = as_pandas(cur)

col_list = [ 'encntr_id', 'encntr_starttime', 'encntr_duration_hrs', 'encntr_endtime', 'encntr_complete_time',
            'age', 'sex', 'race', 'obese', 'smoker', 'prev_rrt', 'on_iv', 'bu-nal',
           'DBP_mean', 'DBP_recent', 'SBP_mean', 'SBP_recent', 
            'MAP_mean', 'MAP_recent', 
             'temp_mean', 'temp_recent', 'SPO2_mean', 'SPO2_recent',
            'RR_mean', 'RR_recent', 'pulse_mean', 'pulse_recent',
            'anticoagulants', 'narcotics', 'narc-ans',
            'antipsychotics', 'chemo', 'dialysis']

dfshell = dfshell.reindex(columns=col_list)
dfshell['age'] = pd.to_numeric(dfshell.age, errors = 'coerce')



In [5]:
dfshell

,encntr_id,encntr_starttime,encntr_duration_hrs,encntr_endtime,encntr_complete_time,age,sex,race,obese,smoker,...,RR_mean,RR_recent,pulse_mean,pulse_recent,anticoagulants,narcotics,narc-ans,antipsychotics,chemo,dialysis
0,111942957,1471290480000,47.750000,1471462380000,2016-08-17 19:33:00,32,F,White,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,111838650,1470914940000,142.766667,1471428900000,2016-08-17 10:15:00,91,M,Other Race,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,111933036,1471149780000,76.450000,1471425000000,2016-08-17 09:10:00,84,F,White,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,111842881,1470960360000,126.316667,1471415100000,2016-08-17 06:25:00,68,M,Unknown,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,109988188,1469044020000,658.383333,1471414200000,2016-08-17 06:10:00,74,M,White,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,111929764,1471104300000,85.166667,1471410900000,2016-08-17 05:15:00,58,M,Other Race,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,111935802,1471179480000,63.033333,1471406400000,2016-08-17 04:00:00,71,M,White,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,111509430,1470190980000,328.533333,1471373700000,2016-08-16 18:55:00,87,M,Asian,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,111874794,1470985260000,100.483333,1471347000000,2016-08-16 11:30:00,59,M,Other Race,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,111886593,1471065900000,76.666667,1471341900000,2016-08-16 10:05:00,46,F,Asian,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
modeltable1 = dfshell.copy() # hours 0 - 12
modeltable2 = dfshell.copy() # hours 12 - 24
modeltable3 = dfshell.copy() # hours 24 - 36

In [7]:
count = 0
for i in xrange(len(modeltable1)):
# for i in xrange(1):
    count += 1
    print "On loop: {0} of {1}".format(count, len(modeltable1))
    modeltable1 = pull_and_writedata_2query(modeltable1, i, 0)

On loop: 1 of 10
On loop: 2 of 10
On loop: 3 of 10
On loop: 4 of 10
On loop: 5 of 10
On loop: 6 of 10
On loop: 7 of 10
On loop: 8 of 10
On loop: 9 of 10
On loop: 10 of 10


In [8]:
count = 0
for i in xrange(len(modeltable2)):
# for i in xrange(1):
    count += 1
    print "On loop: {0} of {1}".format(count, len(modeltable2))
    modeltable2 = pull_and_writedata_2query(modeltable2, i, 12)

On loop: 1 of 10
On loop: 2 of 10
On loop: 3 of 10
On loop: 4 of 10
On loop: 5 of 10
On loop: 6 of 10
On loop: 7 of 10
On loop: 8 of 10
On loop: 9 of 10
On loop: 10 of 10


In [9]:
count = 0
for i in xrange(len(modeltable3)):
# for i in xrange(1):
    count += 1
    print "On loop: {0} of {1}".format(count, len(modeltable3))
    modeltable3 = pull_and_writedata_2query(modeltable3, i, 24)

On loop: 1 of 10
On loop: 2 of 10
On loop: 3 of 10
On loop: 4 of 10
On loop: 5 of 10
On loop: 6 of 10
On loop: 7 of 10
On loop: 8 of 10
On loop: 9 of 10
On loop: 10 of 10


In [10]:
modeltable1

,encntr_id,encntr_starttime,encntr_duration_hrs,encntr_endtime,encntr_complete_time,age,sex,race,obese,smoker,...,RR_mean,RR_recent,pulse_mean,pulse_recent,anticoagulants,narcotics,narc-ans,antipsychotics,chemo,dialysis
0,111942957,1471290480000,47.750000,1471462380000,2016-08-17 19:33:00,32,F,White,0.0,0.0,...,19.333333,20.0,110.000000,110.0,1.0,1.0,0.0,0.0,0.0,0.0
1,111838650,1470914940000,142.766667,1471428900000,2016-08-17 10:15:00,91,M,Other Race,0.0,0.0,...,20.400000,24.0,97.750000,99.0,0.0,1.0,1.0,1.0,0.0,0.0
2,111933036,1471149780000,76.450000,1471425000000,2016-08-17 09:10:00,84,F,White,NaN,0.0,...,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0
3,111842881,1470960360000,126.316667,1471415100000,2016-08-17 06:25:00,68,M,Unknown,NaN,0.0,...,NaN,NaN,NaN,NaN,1.0,1.0,0.0,1.0,0.0,0.0
4,109988188,1469044020000,658.383333,1471414200000,2016-08-17 06:10:00,74,M,White,NaN,0.0,...,NaN,NaN,NaN,NaN,1.0,1.0,0.0,1.0,0.0,0.0
5,111929764,1471104300000,85.166667,1471410900000,2016-08-17 05:15:00,58,M,Other Race,NaN,0.0,...,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0
6,111935802,1471179480000,63.033333,1471406400000,2016-08-17 04:00:00,71,M,White,0.0,0.0,...,19.200000,20.0,67.333333,65.0,0.0,1.0,0.0,0.0,0.0,0.0
7,111509430,1470190980000,328.533333,1471373700000,2016-08-16 18:55:00,87,M,Asian,0.0,0.0,...,17.571429,18.0,78.666667,84.0,0.0,0.0,1.0,0.0,0.0,0.0
8,111874794,1470985260000,100.483333,1471347000000,2016-08-16 11:30:00,59,M,Other Race,0.0,0.0,...,21.500000,18.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
9,111886593,1471065900000,76.666667,1471341900000,2016-08-16 10:05:00,46,F,Asian,0.0,0.0,...,19.000000,20.0,94.000000,99.0,1.0,1.0,0.0,0.0,0.0,0.0


In [11]:
modeltable2

,encntr_id,encntr_starttime,encntr_duration_hrs,encntr_endtime,encntr_complete_time,age,sex,race,obese,smoker,...,RR_mean,RR_recent,pulse_mean,pulse_recent,anticoagulants,narcotics,narc-ans,antipsychotics,chemo,dialysis
0,111942957,1471290480000,47.750000,1471462380000,2016-08-17 19:33:00,32,F,White,0.0,0.0,...,19.333333,20.0,86.333333,78.0,1.0,1.0,0.0,0.0,0.0,0.0
1,111838650,1470914940000,142.766667,1471428900000,2016-08-17 10:15:00,91,M,Other Race,0.0,0.0,...,20.454545,20.0,93.000000,91.0,0.0,1.0,1.0,1.0,0.0,0.0
2,111933036,1471149780000,76.450000,1471425000000,2016-08-17 09:10:00,84,F,White,NaN,0.0,...,NaN,NaN,NaN,NaN,0.0,1.0,1.0,0.0,0.0,0.0
3,111842881,1470960360000,126.316667,1471415100000,2016-08-17 06:25:00,68,M,Unknown,NaN,0.0,...,NaN,NaN,NaN,NaN,1.0,1.0,0.0,1.0,0.0,0.0
4,109988188,1469044020000,658.383333,1471414200000,2016-08-17 06:10:00,74,M,White,NaN,0.0,...,NaN,NaN,NaN,NaN,1.0,1.0,0.0,1.0,0.0,0.0
5,111929764,1471104300000,85.166667,1471410900000,2016-08-17 05:15:00,58,M,Other Race,NaN,0.0,...,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0
6,111935802,1471179480000,63.033333,1471406400000,2016-08-17 04:00:00,71,M,White,0.0,0.0,...,19.500000,18.0,80.750000,78.0,0.0,1.0,0.0,0.0,0.0,0.0
7,111509430,1470190980000,328.533333,1471373700000,2016-08-16 18:55:00,87,M,Asian,0.0,0.0,...,20.500000,22.0,101.600000,109.0,0.0,0.0,1.0,0.0,0.0,0.0
8,111874794,1470985260000,100.483333,1471347000000,2016-08-16 11:30:00,59,M,Other Race,0.0,0.0,...,18.000000,18.0,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0
9,111886593,1471065900000,76.666667,1471341900000,2016-08-16 10:05:00,46,F,Asian,0.0,0.0,...,19.333333,20.0,85.333333,86.0,1.0,1.0,0.0,0.0,0.0,0.0


In [12]:
modeltable3

,encntr_id,encntr_starttime,encntr_duration_hrs,encntr_endtime,encntr_complete_time,age,sex,race,obese,smoker,...,RR_mean,RR_recent,pulse_mean,pulse_recent,anticoagulants,narcotics,narc-ans,antipsychotics,chemo,dialysis
0,111942957,1471290480000,47.750000,1471462380000,2016-08-17 19:33:00,32,F,White,0.0,0.0,...,19.000000,18.0,79.500000,77.0,1.0,1.0,0.0,0.0,0.0,0.0
1,111838650,1470914940000,142.766667,1471428900000,2016-08-17 10:15:00,91,M,Other Race,0.0,0.0,...,17.500000,17.0,92.000000,92.0,0.0,1.0,1.0,1.0,0.0,0.0
2,111933036,1471149780000,76.450000,1471425000000,2016-08-17 09:10:00,84,F,White,NaN,0.0,...,NaN,NaN,NaN,NaN,0.0,1.0,1.0,0.0,0.0,0.0
3,111842881,1470960360000,126.316667,1471415100000,2016-08-17 06:25:00,68,M,Unknown,NaN,0.0,...,NaN,NaN,NaN,NaN,1.0,1.0,0.0,1.0,0.0,0.0
4,109988188,1469044020000,658.383333,1471414200000,2016-08-17 06:10:00,74,M,White,NaN,0.0,...,NaN,NaN,NaN,NaN,1.0,1.0,0.0,1.0,0.0,0.0
5,111929764,1471104300000,85.166667,1471410900000,2016-08-17 05:15:00,58,M,Other Race,NaN,0.0,...,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0
6,111935802,1471179480000,63.033333,1471406400000,2016-08-17 04:00:00,71,M,White,0.0,0.0,...,18.666667,16.0,86.000000,84.0,0.0,1.0,0.0,0.0,0.0,0.0
7,111509430,1470190980000,328.533333,1471373700000,2016-08-16 18:55:00,87,M,Asian,0.0,0.0,...,20.666667,22.0,89.666667,107.0,0.0,1.0,1.0,0.0,0.0,0.0
8,111874794,1470985260000,100.483333,1471347000000,2016-08-16 11:30:00,59,M,Other Race,0.0,0.0,...,18.000000,18.0,79.000000,79.0,1.0,0.0,0.0,0.0,0.0,0.0
9,111886593,1471065900000,76.666667,1471341900000,2016-08-16 10:05:00,46,F,Asian,0.0,0.0,...,17.333333,18.0,84.666667,81.0,1.0,1.0,0.0,0.0,0.0,0.0


### Subset columns & predict for each modelingtable

In [27]:

# binarize sex
modeltable1['is_male'] = modeltable1['sex'].map({'M': 1, 'F': 0})
modeltable1.pop('sex')
modeltable2['is_male'] = modeltable2['sex'].map({'M': 1, 'F': 0})
modeltable2.pop('sex')
modeltable3['is_male'] = modeltable3['sex'].map({'M': 1, 'F': 0})
modeltable3.pop('sex')


0    F
1    M
2    F
3    M
4    M
5    M
6    M
7    M
8    M
9    F
Name: sex, dtype: object

In [28]:
col_use = [ 'age', 'is_male', 'obese', 'smoker', 'prev_rrt', 'on_iv', 'bu-nal',
           'DBP_mean', 'DBP_recent', 'SBP_mean', 'SBP_recent', 
            'MAP_mean', 'MAP_recent', 
             'temp_mean', 'temp_recent', 'SPO2_mean', 'SPO2_recent',
            'RR_mean', 'RR_recent', 'pulse_mean', 'pulse_recent',
            'anticoagulants', 'narcotics', 'narc-ans',
            'antipsychotics', 'chemo', 'dialysis']

In [29]:
X1 = modeltable1[col_use]
X2 = modeltable2[col_use]
X3 = modeltable3[col_use]

In [30]:
# let's look at getting rid of the data rows where vitals signs are all nans
vitals_cols = ['DBP_mean', 'DBP_recent', # take the mean of all the measurements & the most recently observed point
            'SBP_mean', 'SBP_recent',
            'MAP_mean', 'MAP_recent', # mean arterial pressure
             'temp_mean', 'temp_recent',# temperature
             'SPO2_mean', 'SPO2_recent',
            'RR_mean', 'RR_recent', # respiratory rate
            'pulse_mean', 'pulse_recent']

# Which rows are all nans?

print np.where(X1.ix[:, vitals_cols].sum(axis=1, skipna=True)!=0)[0]
print np.where(X2.ix[:, vitals_cols].sum(axis=1, skipna=True)!=0)[0]
print np.where(X3.ix[:, vitals_cols].sum(axis=1, skipna=True)!=0)[0]

[0 1 6 7 8 9]
[0 1 6 7 8 9]
[0 1 6 7 8 9]


In [31]:
# Oh good, they're all the same samples. let's get rid of those rows.
# Write out rows that are not all 0/NaNs across. (if all nans, remove this sample)
X1 = X1.loc[np.where(X1.ix[:, vitals_cols].sum(axis=1, skipna=True)!=0)[0]]; X1 = X1.reset_index(drop=True)
X2 = X2.loc[np.where(X2.ix[:, vitals_cols].sum(axis=1, skipna=True)!=0)[0]]; X2 = X2.reset_index(drop=True)
X3 = X3.loc[np.where(X3.ix[:, vitals_cols].sum(axis=1, skipna=True)!=0)[0]]; X3 = X3.reset_index(drop=True)

# if 'obese' is Nan, then set the patient to be not obese.
X1.loc[np.where(pd.isnull(X1['obese']))[0], 'obese'] = 0
X2.loc[np.where(pd.isnull(X2['obese']))[0], 'obese'] = 0
X3.loc[np.where(pd.isnull(X3['obese']))[0], 'obese'] = 0

In [32]:
X1.T

,0,1,2,3,4,5
age,32.000000,91.00,71.000000,87.000000,59.00,46.000000
is_male,0.000000,1.00,1.000000,1.000000,1.00,0.000000
obese,0.000000,0.00,0.000000,0.000000,0.00,0.000000
smoker,0.000000,0.00,0.000000,0.000000,0.00,0.000000
prev_rrt,0.000000,0.00,0.000000,0.000000,0.00,0.000000
on_iv,1.000000,1.00,0.000000,0.000000,0.00,1.000000
bu-nal,0.000000,0.00,0.000000,0.000000,0.00,0.000000
DBP_mean,64.400000,70.00,82.800000,59.000000,98.50,90.666667
DBP_recent,71.000000,70.00,81.000000,56.000000,99.00,89.000000
SBP_mean,97.000000,120.60,141.800000,108.142857,157.70,160.333333


In [33]:
X2.T

,0,1,2,3,4,5
age,32.000000,91.000000,71.000000,87.00,59.000000,46.000000
is_male,0.000000,1.000000,1.000000,1.00,1.000000,0.000000
obese,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
smoker,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
prev_rrt,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
on_iv,1.000000,1.000000,0.000000,0.00,0.000000,1.000000
bu-nal,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
DBP_mean,70.333333,62.636364,80.500000,48.10,94.000000,91.666667
DBP_recent,70.000000,57.000000,90.000000,57.00,91.000000,92.000000
SBP_mean,110.000000,114.545455,133.000000,83.30,148.750000,158.000000


In [34]:
X3.T

,0,1,2,3,4,5
age,32.00,91.00,71.000000,87.000000,59.000000,46.000000
is_male,0.00,1.00,1.000000,1.000000,1.000000,0.000000
obese,0.00,0.00,0.000000,0.000000,0.000000,0.000000
smoker,0.00,0.00,0.000000,0.000000,0.000000,0.000000
prev_rrt,0.00,0.00,0.000000,0.000000,0.000000,0.000000
on_iv,1.00,1.00,0.000000,1.000000,0.000000,1.000000
bu-nal,0.00,0.00,0.000000,0.000000,0.000000,0.000000
DBP_mean,76.50,72.20,83.250000,54.333333,89.000000,89.666667
DBP_recent,79.00,88.00,89.000000,44.000000,95.000000,92.000000
SBP_mean,115.00,131.60,149.250000,97.000000,136.666667,153.000000


In [35]:
## some entries for temperature & pulse are nan. Let's fill those values, using the mean from the column.
# Fill nans with mean of columns
X1 = X1.fillna(X1.mean())
X2 = X2.fillna(X2.mean())
X3 = X3.fillna(X3.mean())

In [36]:
X1

,age,is_male,obese,smoker,prev_rrt,on_iv,bu-nal,DBP_mean,DBP_recent,SBP_mean,...,RR_mean,RR_recent,pulse_mean,pulse_recent,anticoagulants,narcotics,narc-ans,antipsychotics,chemo,dialysis
0,32,0,0.0,0.0,0.0,1.0,0.0,64.400000,71.0,97.000000,...,19.333333,20.0,110.000000,110.0,1.0,1.0,0.0,0.0,0.0,0.0
1,91,1,0.0,0.0,0.0,1.0,0.0,70.000000,70.0,120.600000,...,20.400000,24.0,97.750000,99.0,0.0,1.0,1.0,1.0,0.0,0.0
2,71,1,0.0,0.0,0.0,0.0,0.0,82.800000,81.0,141.800000,...,19.200000,20.0,67.333333,65.0,0.0,1.0,0.0,0.0,0.0,0.0
3,87,1,0.0,0.0,0.0,0.0,0.0,59.000000,56.0,108.142857,...,17.571429,18.0,78.666667,84.0,0.0,0.0,1.0,0.0,0.0,0.0
4,59,1,0.0,0.0,0.0,0.0,0.0,98.500000,99.0,157.700000,...,21.500000,18.0,89.550000,91.4,0.0,0.0,0.0,0.0,0.0,0.0
5,46,0,0.0,0.0,0.0,1.0,0.0,90.666667,89.0,160.333333,...,19.000000,20.0,94.000000,99.0,1.0,1.0,0.0,0.0,0.0,0.0


In [103]:
### Load saved model
# from sklearn.externals import joblib
gbc = joblib.load('gbc_base_1.compressed')

In [104]:
gbc

GradientBoostingClassifier(init=None, learning_rate=0.05, loss='deviance',
              max_depth=3, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=2, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=300,
              presort='auto', random_state=None, subsample=0.75, verbose=0,
              warm_start=False)

In [40]:
ypred1 = gbc.predict_proba(X1)
ypred2 = gbc.predict_proba(X2)
ypred3 = gbc.predict_proba(X3)

In [41]:
ypred1[:,1]

array([ 0.94838151,  0.95374524,  0.94946722,  0.84962242,  0.96850423,
        0.98499384])

In [42]:
ypred2

array([[ 0.0550046 ,  0.9449954 ],
       [ 0.01615811,  0.98384189],
       [ 0.04148987,  0.95851013],
       [ 0.03832449,  0.96167551],
       [ 0.04499748,  0.95500252],
       [ 0.02448577,  0.97551423]])

In [43]:
ypred3

array([[ 0.03938557,  0.96061443],
       [ 0.03677708,  0.96322292],
       [ 0.04201023,  0.95798977],
       [ 0.0503959 ,  0.9496041 ],
       [ 0.07305626,  0.92694374],
       [ 0.02396977,  0.97603023]])

In [44]:
pred_probs1 = pd.DataFrame(data=ypred1[:,1], columns =["model_probability_of_rrt"])
pred_probs1['model_probability_of_rrt'] = pd.to_numeric(pred_probs1.model_probability_of_rrt)
pred_probs1['score'] = pred_probs1['model_probability_of_rrt'].apply(lambda x: int(round(x*10.0, 0)))

In [45]:
pred_probs2 = pd.DataFrame(data=ypred2[:,1], columns =["model_probability_of_rrt"])
pred_probs2['model_probability_of_rrt'] = pd.to_numeric(pred_probs2.model_probability_of_rrt)
pred_probs2['score'] = pred_probs2['model_probability_of_rrt'].apply(lambda x: int(round(x*10.0, 0)))

In [46]:
pred_probs3 = pd.DataFrame(data=ypred3[:,1], columns =["model_probability_of_rrt"])
pred_probs3['model_probability_of_rrt'] = pd.to_numeric(pred_probs3.model_probability_of_rrt)
pred_probs3['score'] = pred_probs3['model_probability_of_rrt'].apply(lambda x: int(round(x*10.0, 0)))

In [73]:
enc1 = modeltable1.iloc[np.where(modeltable1.ix[:, vitals_cols].sum(axis=1, skipna=True)!=0)[0],:]['encntr_id']
enc1 = enc1.reset_index(drop=True)
enc2 = modeltable2.iloc[np.where(modeltable2.ix[:, vitals_cols].sum(axis=1, skipna=True)!=0)[0],:]['encntr_id']
enc2 = enc2.reset_index(drop=True)
enc3 = modeltable3.iloc[np.where(modeltable3.ix[:, vitals_cols].sum(axis=1, skipna=True)!=0)[0],:]['encntr_id']
enc3 = enc3.reset_index(drop=True)

In [74]:
enc1

0    111942957
1    111838650
2    111935802
3    111509430
4    111874794
5    111886593
Name: encntr_id, dtype: object

In [75]:
df1 = pd.concat([enc1, X1, pred_probs1],axis=1)
df2 = pd.concat([enc2, X2, pred_probs2],axis=1)
df3 = pd.concat([enc3, X3, pred_probs3],axis=1)

In [76]:
df1

,encntr_id,age,is_male,obese,smoker,prev_rrt,on_iv,bu-nal,DBP_mean,DBP_recent,...,pulse_mean,pulse_recent,anticoagulants,narcotics,narc-ans,antipsychotics,chemo,dialysis,model_probability_of_rrt,score
0,111942957,32,0,0.0,0.0,0.0,1.0,0.0,64.400000,71.0,...,110.000000,110.0,1.0,1.0,0.0,0.0,0.0,0.0,0.948382,9
1,111838650,91,1,0.0,0.0,0.0,1.0,0.0,70.000000,70.0,...,97.750000,99.0,0.0,1.0,1.0,1.0,0.0,0.0,0.953745,10
2,111935802,71,1,0.0,0.0,0.0,0.0,0.0,82.800000,81.0,...,67.333333,65.0,0.0,1.0,0.0,0.0,0.0,0.0,0.949467,9
3,111509430,87,1,0.0,0.0,0.0,0.0,0.0,59.000000,56.0,...,78.666667,84.0,0.0,0.0,1.0,0.0,0.0,0.0,0.849622,8
4,111874794,59,1,0.0,0.0,0.0,0.0,0.0,98.500000,99.0,...,89.550000,91.4,0.0,0.0,0.0,0.0,0.0,0.0,0.968504,10
5,111886593,46,0,0.0,0.0,0.0,1.0,0.0,90.666667,89.0,...,94.000000,99.0,1.0,1.0,0.0,0.0,0.0,0.0,0.984994,10


In [88]:
# May need to rename columns to get rid of dash in name...
df1.rename(columns={'bu-nal': 'bu_nal', 'narc-ans': 'narc_ans'}, inplace=True)
df2.rename(columns={'bu-nal': 'bu_nal', 'narc-ans': 'narc_ans'}, inplace=True)
df3.rename(columns={'bu-nal': 'bu_nal', 'narc-ans': 'narc_ans'}, inplace=True)

In [77]:
# Close previous impyla connection, reconnect on ibis...
conn.close()

In [78]:
# Use ibis to save dfs to HDFS
import ibis

In [86]:
hdfs = ibis.hdfs_connect(host="socbdnn01.sharp.com")
con = ibis.impala.connect(host="socbddn01.sharp.com", port=21050, hdfs_client=hdfs)
db = con.database('poc_sandbox')

In [95]:
db.create_table('eys_ExamplePatientsScores_0_12hrs', df1)
t = db.eys_ExamplePatientsScores_0_12hrs
t.execute()

In [99]:
db.create_table('eys_ExamplePatientsScores_12_24hrs', df2)
t = db.eys_ExamplePatientsScores_12_24hrs
t.execute()

,encntr_id,age,is_male,obese,smoker,prev_rrt,on_iv,bu_nal,dbp_mean,dbp_recent,...,pulse_mean,pulse_recent,anticoagulants,narcotics,narc_ans,antipsychotics,chemo,dialysis,model_probability_of_rrt,score
0,111942957,32,0,0.0,0.0,0.0,1.0,0.0,70.333333,70.0,...,86.333333,78.0,1.0,1.0,0.0,0.0,0.0,0.0,0.944995,9
1,111838650,91,1,0.0,0.0,0.0,1.0,0.0,62.636364,57.0,...,93.000000,91.0,0.0,1.0,1.0,1.0,0.0,0.0,0.983842,10
2,111935802,71,1,0.0,0.0,0.0,0.0,0.0,80.500000,90.0,...,80.750000,78.0,0.0,1.0,0.0,0.0,0.0,0.0,0.958510,10
3,111509430,87,1,0.0,0.0,0.0,0.0,0.0,48.100000,57.0,...,101.600000,109.0,0.0,0.0,1.0,0.0,0.0,0.0,0.961676,10
4,111874794,59,1,0.0,0.0,0.0,0.0,0.0,94.000000,91.0,...,89.403333,88.4,1.0,0.0,0.0,0.0,0.0,0.0,0.955003,10
5,111886593,46,0,0.0,0.0,0.0,1.0,0.0,91.666667,92.0,...,85.333333,86.0,1.0,1.0,0.0,0.0,0.0,0.0,0.975514,10


In [100]:
db.create_table('eys_ExamplePatientsScores_24_36hrs', df3)
t = db.eys_ExamplePatientsScores_24_36hrs
t.execute()

,encntr_id,age,is_male,obese,smoker,prev_rrt,on_iv,bu_nal,dbp_mean,dbp_recent,...,pulse_mean,pulse_recent,anticoagulants,narcotics,narc_ans,antipsychotics,chemo,dialysis,model_probability_of_rrt,score
0,111942957,32,0,0.0,0.0,0.0,1.0,0.0,76.500000,79.0,...,79.500000,77.0,1.0,1.0,0.0,0.0,0.0,0.0,0.960614,10
1,111838650,91,1,0.0,0.0,0.0,1.0,0.0,72.200000,88.0,...,92.000000,92.0,0.0,1.0,1.0,1.0,0.0,0.0,0.963223,10
2,111935802,71,1,0.0,0.0,0.0,0.0,0.0,83.250000,89.0,...,86.000000,84.0,0.0,1.0,0.0,0.0,0.0,0.0,0.957990,10
3,111509430,87,1,0.0,0.0,0.0,1.0,0.0,54.333333,44.0,...,89.666667,107.0,0.0,1.0,1.0,0.0,0.0,0.0,0.949604,9
4,111874794,59,1,0.0,0.0,0.0,0.0,0.0,89.000000,95.0,...,79.000000,79.0,1.0,0.0,0.0,0.0,0.0,0.0,0.926944,9
5,111886593,46,0,0.0,0.0,0.0,1.0,0.0,89.666667,92.0,...,84.666667,81.0,1.0,1.0,0.0,0.0,0.0,0.0,0.976030,10


In [93]:
db.list_tables()

['allergy',
 'alt_sel_cat',
 'alt_sel_list',
 'ce_coded_result',
 'ce_date_result',
 'ce_dynamic_label',
 'ce_intake_output_result',
 'ce_med_result',
 'ce_microbiology',
 'clinical_event',
 'code_value',
 'diagnosis',
 'dr_location_changes',
 'dr_med_hist_encntr_distinct_med_admin_hr_cnt',
 'dr_med_hist_encntr_med_admin',
 'dr_med_hist_encntr_med_admin_hr_cnt',
 'dr_med_hist_rrt_event',
 'dr_med_hist_rrt_event_distinct_med_hr_bucket',
 'dr_med_hist_rrt_event_med_hr_bucket',
 'dr_med_hist_rrt_event_med_union_hr_bucket',
 'dr_med_hist_rrt_nonevent',
 'dr_med_hist_rrt_nonevent_distinct_med_hr_bucket',
 'dr_med_hist_rrt_nonevent_med_hr_bucket',
 'dr_med_hist_rrt_nonevent_med_union_hr_bucket',
 'encntr_alias',
 'encntr_loc_hist',
 'encntr_plan_reltn',
 'encntr_prsnl_reltn',
 'encounter',
 'eys_examplepatientsscores_0_12hrs',
 'health_plan',
 'med_admin_event',
 'mltm_category_drug_xref',
 'mltm_drug_categories',
 'mltm_drug_id',
 'mltm_drug_name',
 'nomenclature',
 'non_rrt_events',
 'orde

In [101]:
con.close()

In [ ]:
# from impala.dbapi import connect
# conn = connect(host="socbddn01.sharp.com", port=21050)
# cur = conn.cursor()
# cur.execute("use poc_sandbox")